In [1]:
import scipy.io as scio
import numpy as np
import time
import pandas as pd
from OT_CPD.OtSingleDimStatLib import *
import time
from utilsCPD import *
from OT_CPD.WassersteinChangePointDetectionLib import w2SampClustering 

In [6]:
window = 14
stride = 1
w2ConvFilter = scio.loadmat("OT_CPD/TwoSampConvFilter.mat")["filter2"].flatten()
w2ConvFilter = w2ConvFilter[0::int(np.ceil(len(w2ConvFilter)/(2*window)))]-0.166
w2ConvFilter = w2ConvFilter / np.sum(w2ConvFilter)

In [7]:
import pathlib
import os
str(pathlib.Path(os.getcwd()).parent)

file_name = 'Occupancy.csv'

file_path = os.path.normpath(os.path.join(os.getcwd(),"datasets/Occupancy",file_name))

Occupancy = pd.read_csv(file_path).iloc[:,1:]

Targets = pd.read_csv(os.path.normpath(os.path.join(os.getcwd(),"datasets/Occupancy/OccupancyTargets.csv"))).iloc[:,1:]

GroundTruth = list(Targets.to_numpy().flatten())

Occupancy.head()


data = Occupancy.to_numpy()

N_obs = len(data)

## run with several window sizes


#window_size = [200,500,750,1000]

window_size = [750]
METRICS_OCCUPANCY = {}

for window in window_size:
    METRICS_OCCUPANCY[str(window)] = {}
    res  = w2SampClustering(data,'occupancy',window,stride,11,w2ConvFilter)
    cp_pred = res[2]
    METRICS_OCCUPANCY[str(window)]['F1 & AUC & FP'] = f_measure({'0':GroundTruth},list(cp_pred),margin=30)
    METRICS_OCCUPANCY[str(window)]['DD'] = detection_delay(GroundTruth,cp_pred)[1]
    METRICS_OCCUPANCY[str(window)]['Covering'] = covering({'0':GroundTruth},cp_pred,n_obs=N_obs)

    print(METRICS_OCCUPANCY[str(window)])



OT_Occ_AUCs = [f_measure({'0':GroundTruth},list(cp_pred),margin=tau)[2] for tau in [10,30,50,75,100]]
OT_Occ_AUCs

{'F1 & AUC & FP': (0.4000000000000001, 0.40723981900452494, 11), 'DD': 129.6875, 'Covering': 0.7380780308864959}


c:\Users\Sven Jacob\miniconda3\envs\CodePaper\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\Sven Jacob\miniconda3\envs\CodePaper\Lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (11). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


[13, 11, 10, 9, 8]

In [9]:
import zipfile
import json

zip_file_path = './datasets/MNISTSeq.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_contents = zip_ref.namelist()
    json_file_name = zip_contents[0]
    with zip_ref.open(json_file_name) as json_file:
        data = json.load(json_file)

In [ ]:
F1,Covering,AUC,DD,FP,RT = [],[],[],[],[],[]


window  =150
stride = 1

AUC_5,AUC_10,AUC_15,AUC_30 = [],[],[],[]
FP_5,FP_10,FP_15,FP_30 = [],[],[],[]
Cov_5,Cov_10,Cov_15,Cov_30 = [],[],[],[]

for id, k in enumerate(data.keys()):
    ground_truths = data[k]['target']
    ts = np.array(data[k]['data'])
    n = ground_truths[-1] + 200
    start_time = time.time()
    res  = w2SampClustering(ts,'MNIST',window,stride,11,w2ConvFilter)
    cps = res[2]
    f1, AUC_score, false_alarms = f_measure({'0':ground_truths},cps,20)
    covering_score = covering({'0':ground_truths},cps,n)
    delay = detection_delay(ground_truths,cps)[1]
    end_time =time.time()
    F1.append(f1)
    AUC.append(AUC_score)
    Covering.append(covering_score)
    DD.append(delay)
    FP.append(false_alarms)
    RT.append(end_time-start_time)
    print(AUC_score,delay,false_alarms)

    f1, AUC_score, false_alarms = f_measure({'0':ground_truths},cps,5)
    AUC_5.append(AUC_score)
    FP_5.append(false_alarms)
    f1, AUC_score, false_alarms = f_measure({'0':ground_truths},cps,10)
    AUC_10.append(AUC_score)
    FP_10.append(false_alarms)
    f1, AUC_score, false_alarms = f_measure({'0':ground_truths},cps,15)
    AUC_15.append(AUC_score)
    FP_15.append(false_alarms)
    f1, AUC_score, false_alarms = f_measure({'0':ground_truths},cps,30)
    AUC_30.append(AUC_score)
    FP_30.append(false_alarms)


print(np.mean(AUC))
print(np.std(AUC))
print(np.mean(Covering))
print(np.std(Covering))

In [ ]:
df_TS = load_master_data("./datasets/has2023_master.csv.zip")

def prepare_data(df_TS,channels_start,id):
    initalized = False
    for k,name in zip(df_TS.iloc[id,channels_start:],df_TS.columns[channels_start:]):
        if np.array(k).shape[0]!=0:
            if not initalized:
                data = pd.DataFrame(k,columns=[name]).astype('float32')
                initalized = True
            else:
                data[name] = np.array(k).astype(np.float32)
    if df_TS.iloc[id].group == 'outdoor':
        data = data.iloc[:,:-3]
    GroundTruth = df_TS.iloc[id,5]

    data = (data - data.mean())/(data.std())
    return data,GroundTruth

IDs = [10,14,7,182,225,19,185,33,36,87,88,210,11,20,23,243,247,91,95,96,100,141,91,95,245]


window = 1000
stride = 1


METRICS_HASC = {}
METRICS_HASC_25 ={}
METRICS_HASC_50 ={}
METRICS_HASC_150 ={}
METRICS_HASC_200 ={}

for id in IDs:
    #print(id)
    METRICS_HASC[id]  = {}
    METRICS_HASC_25[id]= {}
    METRICS_HASC_50[id] = {}
    METRICS_HASC_150[id] = {}
    METRICS_HASC_200[id] = {}
    df, GroundTruth = prepare_data(df_TS,8,id)
    #print(GroundTruth)
    if df.shape[0]<=1200:
        print('yes')
        continue
    n_cl = len(GroundTruth) if len(GroundTruth) >0 else 1
    try:
        res  = w2SampClustering(df.to_numpy(),'HASC',window,stride,n_cl,w2ConvFilter)
        print(res)
    except:
        continue
    
    cps = res[2]


    print('CPD:',cps)
    f1, AUC_score, false_alarms = f_measure({'0':GroundTruth},cps,100)
    covering_score = covering({'0':GroundTruth},cps,len(df))
    delay = detection_delay(GroundTruth,cps)[1]
    METRICS_HASC[id]['F1'] =f1
    METRICS_HASC[id]['Covering'] = covering_score
    METRICS_HASC[id]['AUC'] = AUC_score
    METRICS_HASC[id]['DD'] = delay
    METRICS_HASC[id]['FP'] = false_alarms

    #print(METRICS_HASC)

    f1, AUC_score, false_alarms = f_measure({'0':GroundTruth},cps,25)
    covering_score = covering({'0':GroundTruth},cps,len(df))
    delay = detection_delay(GroundTruth,cps)[1]
    METRICS_HASC_25[id]['F1'] =f1
    METRICS_HASC_25[id]['Covering'] = covering_score
    METRICS_HASC_25[id]['AUC'] = AUC_score
    METRICS_HASC_25[id]['DD'] = delay
    METRICS_HASC_25[id]['FP'] = false_alarms


    f1, AUC_score, false_alarms = f_measure({'0':GroundTruth},cps,50)
    covering_score = covering({'0':GroundTruth},cps,len(df))
    delay = detection_delay(GroundTruth,cps)[1]
    METRICS_HASC_50[id]['F1'] =f1
    METRICS_HASC_50[id]['Covering'] = covering_score
    METRICS_HASC_50[id]['AUC'] = AUC_score
    METRICS_HASC_50[id]['DD'] = delay
    METRICS_HASC_50[id]['FP'] = false_alarms


    f1, AUC_score, false_alarms = f_measure({'0':GroundTruth},cps,150)
    covering_score = covering({'0':GroundTruth},cps,len(df))
    delay = detection_delay(GroundTruth,cps)[1]
    METRICS_HASC_150[id]['F1'] =f1
    METRICS_HASC_150[id]['Covering'] = covering_score
    METRICS_HASC_150[id]['AUC'] = AUC_score
    METRICS_HASC_150[id]['DD'] = delay
    METRICS_HASC_150[id]['FP'] = false_alarms

    f1, AUC_score, false_alarms = f_measure({'0':GroundTruth},cps,200)
    covering_score = covering({'0':GroundTruth},cps,len(df))
    delay = detection_delay(GroundTruth,cps)[1]
    METRICS_HASC_200[id]['F1'] =f1
    METRICS_HASC_200[id]['Covering'] = covering_score
    METRICS_HASC_200[id]['AUC'] = AUC_score
    METRICS_HASC_200[id]['DD'] = delay
    METRICS_HASC_200[id]['FP'] = false_alarms
